In [26]:
import os, json
from PIL import Image
from tqdm import tqdm
import numpy as np
import torch
import clip

device = "cuda"

# IMAGES DOWNLOAD

In [2]:
!mkdir -p coco

In [20]:
!curl -L "http://images.cocodataset.org/zips/train2017.zip" -o coco/train2017.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18441M 100 18441M   0     0 28147k     0   0:11:10  0:11:10 --:--:-- 32679k


In [21]:
!unzip -q coco/train2017.zip -d coco

# EMBED USING CLIP VIT/B32

In [28]:
model, preprocess = clip.load("ViT-B/32", device=device)

In [29]:
IMAGE_DIR = "./coco/train2017"
IMG_JSON = "./coco/image_clip_b32.json"

In [30]:
results = []
def embed_batch(images):
    # Preprocess each image. stack into a batch
    imgs = torch.stack([preprocess(img).to(device) for img in images])

    with torch.no_grad():
        vec = model.encode_image(imgs).float()

    # norm for cosine similarity
    vec = vec / vec.norm(dim=-1, keepdim=True)

    return vec.cpu().numpy()


In [31]:
files = sorted(os.listdir(IMAGE_DIR))
batch_imgs = []
batch_paths = []

for fname in tqdm(files):
    if not fname.endswith(".jpg"):
        continue

    img_path = os.path.join(IMAGE_DIR, fname)

    try:
        img = Image.open(img_path).convert("RGB")
        batch_imgs.append(img)
        batch_paths.append(img_path)

        if len(batch_imgs) == 32:
            vecs = embed_batch(batch_imgs)
            for p, v in zip(batch_paths, vecs):
                results.append({
                    "image_path": p,
                    "embedding": v.tolist()
                })
            batch_imgs, batch_paths = [], []  # reset

    except Exception as e:
        print("E:", fname, e)

# Process final partial batch
if batch_imgs:
    vecs = embed_batch(batch_imgs)
    for p, v in zip(batch_paths, vecs):
        results.append({
            "image_path": p,
            "embedding": v.tolist()
        })

  0%|                                                                                                           | 0/118287 [00:00<?, ?it/s]/home/cauman/work/tsne-research/backend/venv/lib/python3.11/site-packages/torch/nn/functional.py:6487: UserWarning: Flash Efficient attention on Current AMD GPU is still experimental. Enable it with TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL=1. (Triggered internally at /pytorch/aten/src/ATen/native/transformers/hip/sdp_utils.cpp:287.)
  attn_output = scaled_dot_product_attention(
/home/cauman/work/tsne-research/backend/venv/lib/python3.11/site-packages/torch/nn/functional.py:6487: UserWarning: Mem Efficient attention on Current AMD GPU is still experimental. Enable it with TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL=1. (Triggered internally at /pytorch/aten/src/ATen/native/transformers/hip/sdp_utils.cpp:338.)
  attn_output = scaled_dot_product_attention(
100%|█████████████████████████████████████████████████████████████████████████████████████████████| 118

In [32]:
with open(IMG_JSON, "w") as f:
    json.dump(results, f)

print("Saved:", IMG_JSON)
print("Total images embedded:", len(results))


Saved: ./coco/image_clip_b32.json
Total images embedded: 118287
